# Build Custom Gates

In this notebook, custom single-qubit gates will be implemented in quil. These gates will be produced using a function that returns the transfer function for an operator whenever x (theta), y (gamma), and z (phi) rotation angles are sent as input. 

To demonstrate that these gates work, a decomposed toffoli operation will be described using the custom gates rather than with built in single-qubit gates.

In [1]:
import pyquil
from pyquil import Program, get_qc
from pyquil.gates import *
from pyquil.api import WavefunctionSimulator
from pyquil.quil import DefGate
import numpy as np
import matplotlib.pyplot as plt

print("pyquil version = %s"%pyquil.__version__)

pyquil version = 2.0.0


Create the function that inputs rotation angles and outputs a transformation function.

In [2]:
def make_transfer_matrix(theta,gamma,phi):
    U_mat = np.array([[np.exp(complex(0,-1)*phi/2)*(np.cos(theta/2)*np.cos(gamma/2)
                                            +complex(0,-1)*np.sin(theta/2)*np.sin(gamma/2)),
                               
                               np.exp(complex(0,1)*phi/2)*(-np.cos(theta/2)*np.sin(gamma/2)
                                            +complex(0,-1)*np.sin(theta/2)*np.cos(gamma/2))],
                              
                              [np.exp(complex(0,-1)*phi/2)*(complex(0,-1)*np.sin(theta/2)*np.cos(gamma/2)
                                            +np.cos(theta/2)*np.sin(gamma/2)),
                               
                               np.exp(complex(0,1)*phi/2)*(complex(0,1)*np.sin(theta/2)*np.sin(gamma/2)
                                            +np.cos(theta/2)*np.cos(gamma/2))]])
    
    return U_mat

Create custom versions of all the gates we will use to decompose the Toffoli gate into single- and double-qubit operators (CNOT gate will still be used in this example).

In [3]:
#hadamard = Rz(pi/2)Rx(pi/2)Rz(pi/2)
Z_pi_rot_cust = make_transfer_matrix(0,0,np.pi/2)
Z_pi_rot_cust_def = DefGate("Z_pi_rot_cust",Z_pi_rot_cust)
Z_pi_rot_CUST = Z_pi_rot_cust_def.get_constructor()

X_pi_rot_cust = make_transfer_matrix(np.pi/2,0,0)
X_pi_rot_cust_def = DefGate("X_pi_rot_cust",X_pi_rot_cust)
X_pi_rot_CUST = X_pi_rot_cust_def.get_constructor()

#T gate
T_cust = make_transfer_matrix(0,0,np.pi/4)
T_cust_def = DefGate("T_cust",T_cust)
T_CUST = T_cust_def.get_constructor()

#T adjoint gate
T_adj_cust = make_transfer_matrix(0,0,-np.pi/4)
T_adj_cust_def = DefGate("T_adj_cust",T_adj_cust)
T_ADJ_CUST = T_adj_cust_def.get_constructor()

#S gate
S_cust = make_transfer_matrix(0,0,np.pi/2)
S_cust_def = DefGate("S_cust",S_cust)
S_CUST = S_cust_def.get_constructor()

#X gate (not needed for Toffoli, but used to create input values)
X_cust = make_transfer_matrix(np.pi,0,0)
X_cust_def = DefGate("X_cust",X_cust)
X_CUST = X_cust_def.get_constructor()


Create the program describing the toffoli gate.

In [4]:
p_toff = Program()
p_toff = p_toff + Z_pi_rot_cust_def + X_pi_rot_cust_def + T_cust_def + T_adj_cust_def + S_cust_def 

p_toff = p_toff + Z_pi_rot_CUST(2) + X_pi_rot_CUST(2) + Z_pi_rot_CUST(2)
p_toff += CNOT(1,2)
p_toff += T_ADJ_CUST(2)
p_toff += CNOT(0,2)
p_toff += T_CUST(2)
p_toff += CNOT(1,2)
p_toff += T_ADJ_CUST(2)
p_toff += CNOT(0,2)
p_toff += T_ADJ_CUST(1)
p_toff += T_CUST(2)
p_toff += CNOT(0,1)
p_toff = p_toff + Z_pi_rot_CUST(2) + X_pi_rot_CUST(2) + Z_pi_rot_CUST(2)
p_toff += T_ADJ_CUST(1)
p_toff += CNOT(0,1)
p_toff += T_CUST(0)
p_toff += S_CUST(1)

Analyze output with wavefunction simulator when input to toffoli gate is qubits[2:0] = 000 (marginal error due to floating point error). Since the controls, qubits 0 and 1, are not equal to |1>, a Pauli-X operation will not occur on the target, qubit 2. The output should be qubits[2:0] = 000.

In [5]:
wf_sim = WavefunctionSimulator()
wfn = wf_sim.wavefunction(p_toff)
out = wfn.get_outcome_probs()
print("Basis: Probability of Measurement")
for item in out:
    print("%s : %s"%(item,out[item]))

Basis: Probability of Measurement
000 : 0.9999999999999998
001 : 0.0
010 : 0.0
011 : 0.0
100 : 6.548161810916602e-32
101 : 0.0
110 : 0.0
111 : 0.0


Now we will change the input to be qubits[2:0] = 110. Since qubits 0 and 1 are equal to |1>, a Pauli-X operator will cause a bit flip on qubit 2! The output should be qubits[2:0] = 111.

In [6]:
new_p_toff = Program(X(0),X(1))
new_p_toff += p_toff
wf_sim2 = WavefunctionSimulator()
wfn2 = wf_sim2.wavefunction(new_p_toff)
out2 = wfn2.get_outcome_probs()
print("Basis: Probability of Measurement")
for item in out2:
    print("%s : %s"%(item,out2[item]))

Basis: Probability of Measurement
000 : 0.0
001 : 0.0
010 : 0.0
011 : 4.082971482100939e-32
100 : 0.0
101 : 0.0
110 : 0.0
111 : 0.9999999999999996
